In [20]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go

In [25]:
for folder in os.listdir('.'):
    if os.path.isdir(folder) and '_results' in folder:
        final_df = pd.DataFrame()
        for file in os.listdir(folder):
            data = pd.read_json(f'{folder}/{file}')
            file_name = file.strip('.jsonl')
            data[file_name] = data['correct'] / (data['correct'] + data['wrong'])
            data.drop(columns=['correct', 'wrong', 'correct_confidence', 'wrong_confidence'], inplace=True)
            if final_df.empty:
                final_df = data
            else:
                # merge based on identity_term 
                final_df = final_df.merge(data, on='identity_term')

        # make a plot which scales the x-axis based on the number of identity terms
        fig = go.Figure()
        for col in final_df.columns[1:]:
            fig.add_trace(go.Scatter
            (
                x=final_df['identity_term'],
                y=final_df[col],
                mode='lines+markers',
                name=col
            ))
        fig.update_layout(title=f'{folder} - Correctness of identity terms', xaxis_title='Identity term', yaxis_title='Correctness')
        fig.show()

        # pivot the data to make another plot
        final_df = final_df.melt(id_vars='identity_term', var_name='model', value_name='correctness')
        fig = px.bar(final_df, x='identity_term', y='correctness', color='model', barmode='group')
        fig.update_layout(title=f'{folder} - Correctness of identity terms', xaxis_title='Identity term', yaxis_title='Correctness')
        fig.show()